<h1>Creazione file per popolamento DB</h1>

Si vogliono creare tre file per il popolamento iniziale del database:
- un file per la collezione <code>disciplines</code>, che contiene le informazioni ricavate da <code>entries_discipline</code> e <code>athletes</code>
- un file per la collezione <code>medals</code>, che mantiene le informazioni ricavate da <code>medals</code> e <code>medals_total</code>
- un file per la collezione <code>events</code>, che mantiene le informazioni ricavate da <code>events</code>

In [1]:
import pandas as pd
athletes = pd.read_csv("dataset/athletes.csv")
entries_discipline = pd.read_csv("dataset/entries_discipline.csv")
medals_total = pd.read_csv("dataset/medals_total.csv")
medals = pd.read_csv("dataset/medals.csv")
events = pd.read_csv("dataset/events.csv")

In [2]:
import json
import numpy as np

class NumpyEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):

            return int(obj)

        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)

        elif isinstance(obj, (np.complex_, np.complex64, np.complex128)):
            return {'real': obj.real, 'imag': obj.imag}

        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()

        elif isinstance(obj, (np.bool_)):
            return bool(obj)

        elif isinstance(obj, (np.void)): 
            return None

        return json.JSONEncoder.default(self, obj)

<h2>Disciplines</h2>

In [3]:
entries_discipline.head()

,Discipline,F,M,Total
0,Alpine Skiing,153,157,310
1,Biathlon,106,107,213
2,Bobsleigh,46,117,163
3,Cross-Country Skiing,149,150,299
4,Curling,59,55,114


Il dataset ha quattro colonne, si vuole creare una collezione di documenti per le discipline in cui in ogni documento (disciplina) c'è la lista di tutti gli atleti di quella disciplina.

In [4]:
pd.merge(entries_discipline, athletes, left_on="Discipline", right_on="discipline", how="inner")

,Discipline,F,M,Total,name,short_name,gender,birth_date,birth_place,birth_country,country,country_code,discipline,discipline_code,residence_place,residence_country,height_m/ft,url
0,Alpine Skiing,153,157,310,ABDI Fayik,ABDI F,Male,1997-10-07,"SAN DIEGO, CA",United States of America,Saudi Arabia,KSA,Alpine Skiing,ALP,DHAHRAN,Saudi Arabia,NaN,../../../en/results/alpine-skiing/athlete-prof...
1,Alpine Skiing,153,157,310,ABEDA Shannon,ABEDA S,Male,1996-05-15,"FORT MCMURRAY, AB",Canada,Eritrea,ERI,Alpine Skiing,ALP,"LETHBRIDGE, AB",Canada,NaN,../../../en/results/alpine-skiing/athlete-prof...
2,Alpine Skiing,153,157,310,AERNI Luca,AERNI L,Male,1993-03-27,CHATEL SAINT DENIS,Switzerland,Switzerland,SUI,Alpine Skiing,ALP,MOLLENS,Switzerland,NaN,../../../en/results/alpine-skiing/athlete-prof...
3,Alpine Skiing,153,157,310,AHMADI Atefeh,AHMADI A,Female,2000-12-23,NaN,NaN,Islamic Republic of Iran,IRI,Alpine Skiing,ALP,NaN,NaN,NaN,../../../en/results/alpine-skiing/athlete-prof...
4,Alpine Skiing,153,157,310,AICHER Emma,AICHER E,Female,2003-11-13,SUNDSVALL,Sweden,Germany,GER,Alpine Skiing,ALP,BERCHTESGADEN,Germany,NaN,../../../en/results/alpine-skiing/athlete-prof...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,Speed Skating,78,87,165,ZAKHAROV Ruslan,ZAKHAROV R,Male,1987-03-24,GORKI,USSR,ROC,ROC,Speed Skating,SSK,KRASNOGORSK,Russian Federation,1.73/5'8'',../../../en/results/speed-skating/athlete-prof...
2885,Speed Skating,78,87,165,ZDRAHALOVA Nikola,ZDRAHALOVA N,Female,1996-04-01,DVUR KRALOVE NAD LABEM,Czech Republic,Czech Republic,CZE,Speed Skating,SSK,ZDAR NAD SAZAVOU,Czech Republic,1.71/5'7'',../../../en/results/speed-skating/athlete-prof...
2886,Speed Skating,78,87,165,ZIOMEK Kaja,ZIOMEK K,Female,1997-08-03,LUBIN,Poland,Poland,POL,Speed Skating,SSK,LUBIN,Poland,1.69/5'6'',../../../en/results/speed-skating/athlete-prof...
2887,Speed Skating,78,87,165,ZUREK Damian,ZUREK D,Male,1999-09-17,TOMASZOW MAZOWIECKI,Poland,Poland,POL,Speed Skating,SSK,TOMASZOW MAZOWIECKI,Poland,1.82/5'11'',../../../en/results/speed-skating/athlete-prof...


In [5]:
json_athletes = {}
list_json_athletes = []
json_entries_discipline = {}
list_json_entries_discipline = []

for row in range(entries_discipline.shape[0]):
    for col in range(entries_discipline.shape[1]):
        json_entries_discipline[entries_discipline.columns[col]] = entries_discipline.iloc[row,col]

    athletes_per_discipline = athletes.loc[athletes['discipline'] == entries_discipline['Discipline'][row]]  

    for athlete in range(athletes_per_discipline.shape[0]):
        for col in range(athletes_per_discipline.shape[1]):
            data = athletes_per_discipline.iloc[athlete,col]
            if isinstance(data, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
                if np.isnan(data):
                            pass
            elif data != data:
                pass
            else:
                json_athletes[athletes_per_discipline.columns[col]] = data
        list_json_athletes.append(json_athletes)
        json_athletes = {}

    json_entries_discipline['athletes'] = list_json_athletes
    list_json_entries_discipline.append(json_entries_discipline)
    
    list_json_athletes = []
    json_entries_discipline = {}

jsonfile = open("disciplines.json","w")
for data in list_json_entries_discipline:
    jsonfile.write(json.dumps(data, indent=3, cls=NumpyEncoder))
jsonfile.close()

Ogni documento nella collezione <code>Discipline</code> ha questo formato:

In [6]:
list_json_entries_discipline[0]

{'Discipline': 'Alpine Skiing',
 'F': 153,
 'M': 157,
 'Total': 310,
 'athletes': [{'name': 'ABDI Fayik',
   'short_name': 'ABDI F',
   'gender': 'Male',
   'birth_date': '1997-10-07',
   'birth_place': 'SAN DIEGO, CA',
   'birth_country': 'United States of America',
   'country': 'Saudi Arabia',
   'country_code': 'KSA',
   'discipline': 'Alpine Skiing',
   'discipline_code': 'ALP',
   'residence_place': 'DHAHRAN',
   'residence_country': 'Saudi Arabia',
   'url': '../../../en/results/alpine-skiing/athlete-profile-n1051423-fayik-abdi.htm'},
  {'name': 'ABEDA Shannon',
   'short_name': 'ABEDA S',
   'gender': 'Male',
   'birth_date': '1996-05-15',
   'birth_place': 'FORT MCMURRAY, AB',
   'birth_country': 'Canada',
   'country': 'Eritrea',
   'country_code': 'ERI',
   'discipline': 'Alpine Skiing',
   'discipline_code': 'ALP',
   'residence_place': 'LETHBRIDGE, AB',
   'residence_country': 'Canada',
   'url': '../../../en/results/alpine-skiing/athlete-profile-n1052177-shannon-abeda.htm

<h2>Medals</h2>

Utilizzando i dataser <code>medals</code> e <code>medals_total</code>, si vuole creare una collezione di documenti in cui ogni documento contiene dati relativi ad un paese partecipante e alle relative medaglie vinte.

In [7]:
medals.head()

,medal_type,medal_code,medal_date,athlete_short_name,athlete_name,athlete_sex,athlete_link,event,country,country_code,discipline,discipline_code
0,Gold,1,2022-02-05 00:00:00.0,ECKHOFF T,ECKHOFF Tiril,X,../../../en/results/biathlon/athlete-profile-n...,Mixed Relay 4x6km (W+M),Norway,NOR,Biathlon,BTH
1,Gold,1,2022-02-05 00:00:00.0,ROEISELAND MO,ROEISELAND Marte Olsbu,X,../../../en/results/biathlon/athlete-profile-n...,Mixed Relay 4x6km (W+M),Norway,NOR,Biathlon,BTH
2,Gold,1,2022-02-05 00:00:00.0,BOE T,BOE Tarjei,X,../../../en/results/biathlon/athlete-profile-n...,Mixed Relay 4x6km (W+M),Norway,NOR,Biathlon,BTH
3,Gold,1,2022-02-05 00:00:00.0,BOE JT,BOE Johannes Thingnes,X,../../../en/results/biathlon/athlete-profile-n...,Mixed Relay 4x6km (W+M),Norway,NOR,Biathlon,BTH
4,Silver,2,2022-02-05 00:00:00.0,CHEVALIER-BOUCHET,CHEVALIER-BOUCHET Anais,X,../../../en/results/biathlon/athlete-profile-n...,Mixed Relay 4x6km (W+M),France,FRA,Biathlon,BTH


In [8]:
medals_total.head()

,Order,Country,Gold,Silver,Bronze,Total,Order by Total,Country Code
0,1,Norway,9,5,7,21,1,NOR
1,2,Germany,8,5,1,14,3,GER
2,3,United States of America,6,5,1,12,6,USA
3,4,Netherlands,6,4,2,12,6,NED
4,5,Sweden,5,3,3,11,8,SWE


Per ogni riga di <code>medals_totals</code> (che è relativa a un determinato paese), si crea un documento che ha come campi le colonne di <code>medals</code> più un campo per ogni tipo di medaglia che conterra un array di oggetti per ogni medaglia di quel tipo vinta dal paese.

In [9]:
json_medals_total = {}
list_json_medals_total = []

for row in range(medals_total.shape[0]):
    for col in range(medals_total.shape[1]):
        json_medals_total[medals_total.columns[col]] = medals_total.iloc[row,col]

    medals_country = medals.loc[medals['country_code'] == medals_total['Country Code'][row]]  

    list_gold_medals = []
    list_silver_medals = []
    list_bronze_medals = []

    json_medal = {}

    for medal in range(medals_country.shape[0]):
        for col in range(medals_country.shape[1]):
            data = medals_country.iloc[medal,col]
            if isinstance(data, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
                if np.isnan(data):
                            pass
            elif data != data:
                pass
            else:
                json_medal[medals_country.columns[col]] = data
        if medals_country.iloc[medal,1] == 1:    # oro
            list_gold_medals.append(json_medal)
        elif medals_country.iloc[medal,1] == 2: #argento
            list_silver_medals.append(json_medal)
        else:
            list_bronze_medals.append(json_medal)
        
        json_medal = {}

    if len(list_gold_medals) > 0:
        json_medals_total["gold_medals"] = list_gold_medals
    if len(list_silver_medals) > 0:
        json_medals_total["silver_medals"] = list_silver_medals
    if len(list_bronze_medals) > 0:
        json_medals_total["bronze_medals"] = list_bronze_medals

    list_json_medals_total.append(json_medals_total)
    json_medals_total = {}

jsonfile = open("medals.json","w")
for data in list_json_medals_total:
    jsonfile.write(json.dumps(data, indent=3, cls=NumpyEncoder))
jsonfile.close()

<h2>Events</h2>

In [10]:
events.head()

,location,event_stage,event_status,time,event_url,discipline_code,discipline
0,Yanqing National Alpine Skiing Centre - Speed,Men's Downhill 1st Training,Finished,2022-02-03T11:00:00+08:00,../../../en/results/alpine-skiing/results-men-...,ALP,Alpine Skiing
1,Yanqing National Alpine Skiing Centre - Speed,Men's Downhill 2nd Training,Finished,2022-02-04T12:00:00+08:00,../../../en/results/alpine-skiing/results-men-...,ALP,Alpine Skiing
2,Yanqing National Alpine Skiing Centre - Technical,Women's Giant Slalom Run 1,Finished,2022-02-07T09:30:00+08:00,../../../en/results/alpine-skiing/results-wome...,ALP,Alpine Skiing
3,Yanqing National Alpine Skiing Centre - Speed,Men's Downhill,Finished,2022-02-07T12:00:00+08:00,../../../en/results/alpine-skiing/results-men-...,ALP,Alpine Skiing
4,Yanqing National Alpine Skiing Centre - Technical,Women's Giant Slalom Run 2,Finished,2022-02-07T14:45:00+08:00,../../../en/results/alpine-skiing/results-wome...,ALP,Alpine Skiing


Nel dataset ci sono righe con campi Nan:

In [11]:
events.columns[events.isna().any()].tolist()

['event_status', 'time']

In [12]:
events.loc[events["event_status"].isna()]

,location,event_stage,event_status,time,event_url,discipline_code,discipline
15,Yanqing National Alpine Skiing Centre - Speed,Women's Downhill 3rd Training,NaN,2022-02-14T13:00:00+08:00,../../../en/results/alpine-skiing/results-wome...,ALP,Alpine Skiing
16,Yanqing National Alpine Skiing Centre - Speed,Women's Downhill,NaN,2022-02-15T11:00:00+08:00,../../../en/results/alpine-skiing/results-wome...,ALP,Alpine Skiing
17,Yanqing National Alpine Skiing Centre - Technical,Men's Slalom Run 1,NaN,2022-02-16T10:15:00+08:00,../../../en/results/alpine-skiing/results-men-...,ALP,Alpine Skiing
18,Yanqing National Alpine Skiing Centre - Speed,Women's Alpine Combined Downhill 1st Training,NaN,2022-02-16T10:30:00+08:00,../../../en/results/alpine-skiing/results-wome...,ALP,Alpine Skiing
19,Yanqing National Alpine Skiing Centre - Technical,Men's Slalom Run 2,NaN,2022-02-16T13:45:00+08:00,../../../en/results/alpine-skiing/results-men-...,ALP,Alpine Skiing
...,...,...,...,...,...,...,...
607,National Speed Skating Oval,Men's 1000m,NaN,2022-02-18T16:30:00+08:00,../../../en/results/speed-skating/results-men-...,SSK,Speed Skating
608,National Speed Skating Oval,Men's Mass Start Semifinals,NaN,2022-02-19T15:00:00+08:00,../../../en/results/speed-skating/results-men-...,SSK,Speed Skating
609,National Speed Skating Oval,Women's Mass Start Semifinals,NaN,2022-02-19T15:45:00+08:00,../../../en/results/speed-skating/results-wome...,SSK,Speed Skating
610,National Speed Skating Oval,Men's Mass Start Final,NaN,2022-02-19T16:30:00+08:00,../../../en/results/speed-skating/results-men-...,SSK,Speed Skating


Queste corrispondono ad eventi non ancora disputati.

In [13]:
json_events = {}
list_json_events = []

for row in range(events.shape[0]):
    for col in range(events.shape[1]):
        data = events.iloc[row,col]
        if isinstance(data, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
                if np.isnan(data):
                            pass
        elif data != data:
            pass
        else:
           json_events[events.columns[col]] = data
    
    list_json_events.append(json_events)
    json_events = {}

jsonfile = open("events.json","w")
for data in list_json_events:
    jsonfile.write(json.dumps(data, indent=3, cls=NumpyEncoder))
jsonfile.close()